In [601]:
import pandas as pd
import xgboost
import numpy as np
import seaborn as sea
from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA
# from scipy.linalg import svd

In [602]:
train = pd.read_csv("train.csv")
train = train.drop('Id', axis=1)
finalTest = pd.read_csv("test.csv")
finalTest = finalTest.drop('Id', axis=1)
trainY = train["Category"]
train = train.drop("Category", axis=1)

final = False
dropThreshold = 110
dropThresholdUp = 398
normalize = True
pcaEnable = False
RangPCA = 200

def apply_PCA(train, test):
    X = pd.concat([train, test])
    # X = X.to_numpy()
    pca = PCA(n_components = RangPCA)#'mle')
    pca.fit(X)
    # print(pca.singular_values_)
    return pca.transform(train), \
        pca.transform(test), pca
    
    
def apply_index(dataset):
    ins = pd.read_csv("index.txt", dtype=int)
    indexes = list(map(lambda s : int(s) + 1, ins.columns))
    for column in dataset:
        words = column.split("_")
        if len(words) > 1:
            id = int(words[1])
            if not id in indexes:
                dataset = dataset.drop(f"x_{id}", axis=1)
    return dataset

In [603]:
def erase_features(data, down, up):
    for column in data:
        columnWords = column.split("_")
        # print(columnWords)
        if len(columnWords) > 1 and int(columnWords[1]) > down \
            and int(columnWords[1]) < up:
            data = data.drop(column, axis = 1)
        elif len(columnWords) and normalize > 1:
            std = data[column].std()
            mean = data[column].mean()
            data[column] = (data[column] - mean) / std
    return data

if pcaEnable:
    train, finalTest, pca = apply_PCA(train, finalTest)
else:
    # train = apply_index(train)
    train = erase_features(train, dropThreshold, dropThresholdUp)
    
np.shape(train)

(10000, 200)

In [604]:
if not final:
    x_train, x_test, y_train, y_test = train_test_split(train, trainY, test_size=0.1, random_state=2)
else:
    x_train = train
    y_train = trainY

In [605]:
model = xgboost.XGBClassifier(n_estimators=10000, max_depth=1, learning_rate=0.8, objective='binary:logistic')
model.fit(x_train, y_train)

KeyboardInterrupt: 

In [ ]:
if not final:
     res = model.predict(x_test)

     def score(res : np.ndarray, answ : np.ndarray):
          return len(list(filter(lambda x: x == 0, res - answ))) / np.size(res)

     print(score(np.array(res), np.array(y_test)))
     

0.043


In [ ]:
res = []
if final:
    if pcaEnable:
        res = model.predict(finalTest)
    else:
        test = pd.read_csv("test.csv")
        test = erase_features(test, dropThreshold, dropThresholdUp)
        res = model.predict(test.drop("Id", axis=1))

            

In [ ]:
if final:
    file = open("result.csv", 'w')
    file.write("Id,Category\n")
    for i in range(len(res)):
        file.write(f"{i},{res[i]}\n")
    file.close()